In [110]:
import datetime
import zoneinfo
import email
from email.parser import BytesParser
from email import policy
filepath = 'Your Potential Timetable.eml'
import re
import uuid
from tkinter import *
from tkinter import ttk

In [111]:
with open(filepath) as email_file:
    email_message = email.message_from_file(email_file)
content = (email_message.get_payload())

In [112]:
content1 = content.replace('=\n','')
new_p = re.compile('<td.*?>.*?</td>')

In [113]:
lst1 = new_p.findall(content1)
# print(lst1)
lst1 = lst1[:-1]
# print(len(lst1))

In [114]:
clear_p = re.compile('</*?td.*?>')
# clear_p2 = re.compile('</td>')
for i in range(len(lst1)):
    lst1[i] = re.sub(clear_p,'',lst1[i])
# print(lst1)

In [115]:
# reshape the list to a 2d array
lst2 = []
for i in range(9):
    lst2.append([])
    for j in range(len(lst1)//9):
        lst2[i].append(lst1[9*j+i])

In [116]:
# sort all the codes in list
# lst = ['status','class','section','subject','course','seats open','duration&locations','dates','campus']
# sometime one course only have one duration, to fix this, add another layer of structure
# sometime eml has different form, try to fix it
# still need more form from different email
time_loc = lst2[6]
for i in range(len(time_loc)):
    if "<br />" in time_loc[i]:
        time_loc[i] = time_loc[i].split('<br />')
    elif "<br>" in time_loc[i]:
        time_loc[i] = time_loc[i].split('<br>')
    elif "<br/>" in time_loc[i]:
        time_loc[i] = time_loc[i].split('<br/>')
    else:
        time_loc[i] = [time_loc[i]]
    
# print(time_loc)
# time_loc[i][j]

In [117]:
dura = lst2[7]
for i in range(len(dura)):
    if "<br />" in dura[i]:
        dura[i] = dura[i].split('<br />')
    elif "<br>" in dura[i]:
        dura[i] = dura[i].split('<br>')
    elif "<br/>" in dura[i]:
        dura[i] = dura[i].split('<br/>')
    else:
        dura[i] = [dura[i]]


In [118]:
print(dura)

[['3/02/2025 - 28/04/2025', '5/02/2025 - 30/04/2025', '6/02/2025 - 1/05/2025'], ['3/02/2025 - 28/04/2025', '5/03/2025 - 30/04/2025', '26/02/2025 - 26/02/2025', '5/02/2025 - 19/02/2025'], ['6/02/2025 - 1/05/2025'], ['6/02/2025 - 1/05/2025'], ['24/04/2025 - 24/04/2025', '10/04/2025 - 10/04/2025', '27/03/2025 - 27/03/2025', '13/03/2025 - 13/03/2025', '27/02/2025 - 27/02/2025', '13/02/2025 - 13/02/2025'], ['17/03/2025 - 14/04/2025', '28/04/2025 - 28/04/2025', '3/02/2025 - 3/03/2025', '22/04/2025 - 22/04/2025', '11/03/2025 - 11/03/2025'], ['14/04/2025 - 14/04/2025', '31/03/2025 - 31/03/2025', '17/03/2025 - 17/03/2025', '3/03/2025 - 3/03/2025', '17/02/2025 - 17/02/2025', '3/02/2025 - 3/02/2025'], ['24/03/2025 - 24/03/2025']]


In [119]:
# start time & end time -> datetime.datetime
def duration(duradate:str):
    duradate = duradate.split(' - ')
    start = datetime.datetime.strptime(duradate[0], '%d/%m/%Y')
    end = datetime.datetime.strptime(duradate[1], '%d/%m/%Y')
    return(start,end)

dura_date = []
for i in range(len(dura)):
    for j in range(len(dura[i])):
        dura[i][j] = duration(dura[i][j])

# print(dura)
# dura[i][j][0] = starttime,dura[i][j][1] = endtime
# i = course,j = time duration

In [120]:
# print(time_loc[0][0])

In [121]:
# seperate time & location
p2 = r"^([MTWFS][a-z]*)\s([\d:amp\s]+)-([\d:amp\s]+)-\s(.+)$"
match = re.match(p2,time_loc[0][0])
# print(match.group(3))

In [122]:

weeks = []
time_start = []
time_end = []
location = []
for i in range(len(time_loc)):
    weeks.append([])
    time_start.append([])
    time_end.append([])
    location.append([])
    for j in range(len(time_loc[i])):
        match = re.match(p2,time_loc[i][j])
        weeks[i].append(match.group(1))
        time_start[i].append(match.group(2).strip())
        time_end[i].append(match.group(3).strip())
        location[i].append(match.group(4))


In [123]:
# start = datetime.datetime(2021,2,28,17,0,0)
# start.strftime('%Y%m%dT%H%M%S')

In [124]:
# create event for one course
def create_event(course:str,start_date,end_date,start_time,end_time,location,campus):
    day = start_date
    course_events = ''
    while day <= end_date:
        event = "\nBEGIN:VEVENT"
        event += f"\nSUMMARY:{course}"
        start = datetime.datetime.strptime((str(day.date())+' '+start_time),'%Y-%m-%d %I:%M%p')
        event += f"\nDTSTART:{start.strftime('%Y%m%dT%H%M%S')}"
        end = datetime.datetime.strptime((str(day.date())+' '+end_time),'%Y-%m-%d %I:%M%p')
        event += f"\nDTEND:{end.strftime('%Y%m%dT%H%M%S')}"
        event += f"\nUID:{uuid.uuid4()}"
        event += f"\nCREATED:{datetime.datetime.now().strftime('%Y%m%dT%H%M%S')}"
        event += f"\nDESCRIPTION:{campus}"
        event += f"\nLOCATION:{location}"
        day += datetime.timedelta(days = 7)
        event += "\nEND:VEVENT"
        course_events += event
    return course_events

In [125]:
with open('Your_timetable.ics','w') as f:
    cal = 'BEGIN:VCALENDAR'
    for i in range(len(lst2[3])):
        for j in range(len(dura[i])):
            cal += create_event(lst2[3][i]+' '+ lst2[4][i],dura[i][j][0],dura[i][j][1],time_start[i][j],time_end[i][j],location[i][j],lst2[8][i])
    cal += '\nEND:VCALENDAR'
    f.write(cal)

In [126]:
#cal = create_event(df['subject'][0]+df['course'][0],dura[0][0][0],dura[0][0][1],time_start[0][0],time_end[0][0],location[0][0],df['campus'][0])
#with open('Your_timetable.ics','wb') as f:
#    f.write(cal.to_ical())